In [35]:
from __future__ import division
import imp
import uproot
import matplotlib
#matplotlib.use('agg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import os
import ROOT
import time
import math
from array import array
import collections

lib_function_dir = '/home/tmettler/Desktop/uBoone/do_plots/'

# helper functions
globale = imp.load_source('globale',lib_function_dir+'globale.py')
NuCC = imp.load_source('NuCC_function',lib_function_dir+'NuCC_function.py')
NuCC_w = imp.load_source('NuCCWeight_function',lib_function_dir+'NuCCWeight_function.py')

FF = imp.load_source('FF_functions',lib_function_dir+'FF_functions.py')

N_tot =  4.10331109202e+31   flux=  1.58051463543e+11 for pot:  2.144e+20


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [36]:
#define binning
mom_bins = {}
mom_bins[0] = [ 0.00, 0.18, 0.30, 0.45, 0.77, 2.50 ]
mom_bins[1] = [ 0.00, 0.18, 0.30, 0.45, 0.77, 2.50 ]
mom_bins[2] = [ 0.00, 0.18, 0.30, 0.45, 0.77, 2.50 ]
mom_bins[3] = [ 0.00, 0.30, 0.45, 0.77, 2.50 ]
mom_bins[4] = [ 0.00, 0.30, 0.45, 0.77, 2.50 ]
mom_bins[5] = [ 0.00, 0.30, 0.45, 0.77, 2.50 ]
mom_bins[6] = [ 0.00, 0.30, 0.45, 0.77, 1.28, 2.50 ]
mom_bins[7] = [ 0.00, 0.30, 0.45, 0.77, 1.28, 2.50 ]
mom_bins[8] = [ 0.00, 0.30, 0.45, 0.77, 1.28, 2.50 ]

theta_bins = [ -1.00, -0.50, 0.00, 0.28, 0.47, 0.63, 0.765, 0.865, 0.935, 1.00 ]
len_theta = len(theta_bins) - 1
len_mom = {}
sum_bins = 0
for i in range(len_theta):
    len_mom[i] = len(mom_bins[i]) - 1
    #print len_mom[i]
    sum_bins+=len_mom[i]

In [38]:
#store all histograms needed in one file: bkg, smearing, covariance
from ROOT import gROOT
if(1):
    # pot here = 7.
    path_detsys = '/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted_improved/'

    f_detcv = ROOT.TFile.Open(path_detsys+'xsec_detsys_CV_2D_t_extra/xsec_detsys_CV_2D.root', 'read')
    f_dedx = ROOT.TFile.Open(path_detsys+'xsec_detsys_dEdx_2D_t_extra/xsec_detsys_dEdx_2D.root', 'read')
    f_LYatt = ROOT.TFile.Open(path_detsys+'xsec_detsys_LYAtt_2D_t_extra/xsec_detsys_LYAtt_2D.root', 'read')
    f_LYdown = ROOT.TFile.Open(path_detsys+'xsec_detsys_LYdown_2D_t_extra/xsec_detsys_LYdown_2D.root', 'read')
    f_LYray = ROOT.TFile.Open(path_detsys+'xsec_detsys_LYRay_2D_t_extra/xsec_detsys_LYRayleigh_2D.root', 'read')
    f_recomb2 = ROOT.TFile.Open(path_detsys+'xsec_detsys_recomb2_2D_t_extra/xsec_detsys_recomb2_2D.root', 'read')
    f_sce = ROOT.TFile.Open(path_detsys+'xsec_detsys_SCE_2D_t_extra/xsec_detsys_SCE_2D.root', 'read')
    f_waxz = ROOT.TFile.Open(path_detsys+'xsec_detsys_WireAngleXZ_2D_t_extra/xsec_detsys_WireAngleXZ_2D.root', 'read')
    f_wayz = ROOT.TFile.Open(path_detsys+'xsec_detsys_WireAngleYZ_2D_t_extra/xsec_detsys_WireAngleYZ_2D.root', 'read')
    f_wmx = ROOT.TFile.Open(path_detsys+'xsec_detsys_WireModX_2D_t_extra/xsec_detsys_WireModX_2D.root', 'read')
    f_wmyz = ROOT.TFile.Open(path_detsys+'xsec_detsys_WireModYZ_2D_t_extra/xsec_detsys_WireModYZ_2D.root', 'read')
    
    # pot here = 7.
    f_all = ROOT.TFile.Open(path_detsys+'xsec_detsys_all_2D_t_extra/Detector_systematic_2D.root', 'read')
    
    #h_true_cv = f_cv.Get('h_gen_all')
    #h_sel_cv = f_cv.Get('h_sel_all')

    #h_ext_cv = f_cv.Get('h_ext_all')
    #h_dirt_cv = f_cv.Get('h_dirt_all')


    h_bkg_detcv = f_detcv.Get('h_bkg_all')
    h_bkg_dedx = f_dedx.Get('h_bkg_all')
    h_bkg_LYatt = f_LYatt.Get('h_bkg_all')
    h_bkg_LYdown = f_LYdown.Get('h_bkg_all')
    h_bkg_LYray = f_LYray.Get('h_bkg_all')
    h_bkg_recomb2 = f_recomb2.Get('h_bkg_all')
    h_bkg_sce = f_sce.Get('h_bkg_all')
    h_bkg_waxz = f_waxz.Get('h_bkg_all')
    h_bkg_wayz = f_wayz.Get('h_bkg_all')
    h_bkg_wmx = f_wmx.Get('h_bkg_all')
    h_bkg_wmyz = f_wmyz.Get('h_bkg_all')

    h_smear_detcv = f_detcv.Get('h_smearing')
    h_smear_dedx = f_dedx.Get('h_smearing')
    h_smear_LYatt = f_LYatt.Get('h_smearing')
    h_smear_LYdown = f_LYdown.Get('h_smearing')
    h_smear_LYray = f_LYray.Get('h_smearing')
    h_smear_recomb2 = f_recomb2.Get('h_smearing')
    h_smear_sce = f_sce.Get('h_smearing')
    h_smear_waxz = f_waxz.Get('h_smearing')
    h_smear_wayz = f_wayz.Get('h_smearing')
    h_smear_wmx = f_wmx.Get('h_smearing')
    h_smear_wmyz = f_wmyz.Get('h_smearing')
    
    # cv file ###################################################
    # pot here = 2.144e+20
    f_cv = ROOT.TFile.Open('/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted_improved/xsec_filesV4/xsec_2D_Run3_wighted_t.root', 'read')
    h_ext = ROOT.TH1F('h_ext',"Cosmic, 2.4POT",43,0,43)
    h_dirt = ROOT.TH1F('h_dirt',"Dirt, 2.4POT",43,0,43)
    h_bkg = ROOT.TH1F('h_bkg',"Bkg, 2.4POT",43,0,43)
    h_data = ROOT.TH1F('h_data',"Data, 2.4POT",42,0,42)
    
    h_dirt_nocrt = ROOT.TH1F('h_dirt',"Dirt wo crt, 2.4POT",43,0,43)
    h_bkg_nocrt = ROOT.TH1F('h_bkg',"Bkg wo crt, 2.4POT",43,0,43)
    
    bin_counter = 0
    for t_bin in range(len_theta): #len_theta
        h_temp1 = f_cv.Get('h_data['+str(t_bin)+']')
        h_temp2 = f_cv.Get('h_ext['+str(t_bin)+']')
        h_temp3 = f_cv.Get('h_dirt['+str(t_bin)+']')
        h_temp4 = f_cv.Get('h_overlay['+str(t_bin)+']')
        h_temp5 = f_cv.Get('h_overlay_nocrt['+str(t_bin)+']')
        h_temp6 = f_cv.Get('h_dirt_nocrt['+str(t_bin)+']')
        for m_bin in range(len_mom[t_bin]):
            bin_counter+=1
            h_data.SetBinContent(bin_counter,h_temp1.GetBinContent(m_bin+1))
            h_ext.SetBinContent(bin_counter,h_temp2.GetBinContent(m_bin+1))
            h_dirt.SetBinContent(bin_counter,h_temp3.GetBinContent(m_bin+1))
            h_bkg.SetBinContent(bin_counter,h_temp4.GetBinContent(m_bin+1))
            h_bkg_nocrt.SetBinContent(bin_counter,h_temp5.GetBinContent(m_bin+1))
            h_dirt_nocrt.SetBinContent(bin_counter,h_temp6.GetBinContent(m_bin+1))
    
    h_sel = f_cv.Get('h_sel_all')
    h_gen = f_cv.Get('h_gen_all')
    h_eff = f_cv.Get('h_eff_all')
    h_sel.Scale(2.144e+20/1.26812e+21)
    h_gen.Scale(2.144e+20/1.26812e+21)
    
    h_smearing = f_cv.Get('h_smearing')
    h_smearing_nocrt = f_cv.Get('h_smearing_nocrt')
    
    ### other genie file ################################
    path_other = '/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted_improved/pc46_data/sep24/xsec_genie_other_2D_mar18/root_output/'
    f_other = ROOT.TFile.Open(path_other+'Other_Genie_2D.root', 'read')
    weight_list = [ 'AxFFCCQEshape_UBGenie' , 'DecayAngMEC_UBGenie', 'NormCCCOH_UBGenie', 'NormNCCOH_UBGenie',\
               'RPA_CCQE_UBGenie','ThetaDelta2NRad_UBGenie','Theta_Delta2Npi_UBGenie',\
                'VecFFCCQEshape_UBGenie','XSecShape_CCMEC_UBGenie']
    w_list = [0,1,2,3,4,5,6,7,8]
    
    h_bkg_other = []
    h_bkg_other.append([])
    h_bkg_other.append([])
    h_smear_other = []
    h_smear_other.append([])
    h_smear_other.append([])

    for w in w_list:
        #take first uiverse as cv
        h_bkg_0 = f_other.Get('h_bkg_all['+str(w)+'][0]')
        h_bkg_1 = f_other.Get('h_bkg_all['+str(w)+'][1]')

        bin_counter = 0
        for t_bin in range(len_theta): #len_theta
            h_temp1 = f_other.Get('h_overlay['+str(w)+'][0]['+str(t_bin)+']')
            h_temp2 = f_other.Get('h_overlay['+str(w)+'][1]['+str(t_bin)+']')
            for m_bin in range(len_mom[t_bin]):
                bin_counter+=1
                h_bkg_0.SetBinContent(bin_counter,h_temp1.GetBinContent(m_bin+1))
                h_bkg_1.SetBinContent(bin_counter,h_temp2.GetBinContent(m_bin+1))

        h_smear_0 = f_other.Get('h_smearing['+str(w)+'][0]')
        h_eff_0 = f_other.Get('h_eff_all['+str(w)+'][0]')
        h_eff_0.SetBinContent(5,1)
        h_eff_0.SetBinContent(10,1)

        h_smear_1 = f_other.Get('h_smearing['+str(w)+'][1]')
        h_eff_1 = f_other.Get('h_eff_all['+str(w)+'][1]')
        h_eff_1.SetBinContent(5,1)
        h_eff_1.SetBinContent(10,1)
        
        smear0 = FF.plot2arr(h_smear_0,2)
        smear1 = FF.plot2arr(h_smear_1,2)
        eff0 = FF.plot2arr(h_eff_0,1)
        eff1 = FF.plot2arr(h_eff_1,1)
        for i in range(43):
            for j in range(43):
                smear0[i][j] = smear0[i][j]*eff0[j]
                smear1[i][j] = smear1[i][j]*eff1[j]
                
        h_bkg_other[0].append(h_bkg_0)
        h_bkg_other[1].append(h_bkg_1)
        h_smear_other[0].append(FF.arr2plot(smear0))
        h_smear_other[1].append(FF.arr2plot(smear1))
            
    ### all genie: ######################################
    path_all = '/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted_improved/pc46_data/sep24/xsec_genie_all_0_2D_sep24/root_output/'
    f_all = ROOT.TFile.Open(path_all+'All_Genie_2D.root', 'read')
    
    h_bkg_all_cv = f_all.Get('h_bkg_all[0]')
    bin_counter=0
    for t_bin in range(len_theta): #len_theta
        h_temp = f_all.Get('h_overlay_cv['+str(t_bin)+']')
        for m_bin in range(len_mom[t_bin]):
            bin_counter+=1
            h_bkg_all_cv.SetBinContent(bin_counter,h_temp.GetBinContent(m_bin+1))

    h_smearing_all_cv = f_all.Get('h_smearing_cv')
    h_eff_all_cv = f_all.Get('h_eff_all_cv')
    h_eff_all_cv.SetBinContent(5,1)
    h_eff_all_cv.SetBinContent(10,1)
    for j in range(43):
        for k in range(43):
            h_smearing_all_cv.SetBinContent(j+1,k+1,h_smearing_all_cv.GetBinContent(j+1,k+1)*h_eff_all_cv.GetBinContent(k+1))
    
    h_bkg_all = []
    h_smearing_all = []
    h_eff_all = []
    for uni in range(100):
        #del gROOT.FindObject("h_this")
        tmp = gROOT.FindObject("h_this")
        del tmp
        h_bkg_all.append(h_bkg_all_cv.Clone())
        bin_counter=0
        for t_bin in range(len_theta): #len_theta
            h_temp = f_all.Get('h_overlay['+str(uni)+']['+str(t_bin)+']')
            for m_bin in range(len_mom[t_bin]):
                bin_counter+=1
                h_bkg_all[uni].SetBinContent(bin_counter,h_temp.GetBinContent(m_bin+1))
        h_smearing_tmp = f_all.Get('h_smearing['+str(uni)+']')
        h_eff_tmp = f_all.Get('h_eff_all['+str(uni)+']')
        h_eff_tmp.SetBinContent(5,1)
        h_eff_tmp.SetBinContent(10,1)
        smear = FF.plot2arr(h_smearing_tmp,2)
        eff = FF.plot2arr(h_eff_tmp,1)
        for j in range(43):
            for k in range(43):
                smear[j][k] = smear[j][k]*eff[k]
        h_smearing_all.append(FF.arr2plot(smear))
    
    path_all1 = '/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted_improved/pc46_data/sep24/xsec_genie_all_1_2D_sep24/root_output/'
    f_all1 = ROOT.TFile.Open(path_all1+'All_Genie_2D.root', 'read')
    
    for uni in range(100):
        h_bkg_all.append(h_bkg_all_cv.Clone())
        bin_counter=0
        for t_bin in range(len_theta): #len_theta
            h_temp = f_all1.Get('h_overlay['+str(uni)+']['+str(t_bin)+']')
            for m_bin in range(len_mom[t_bin]):
                bin_counter+=1
                h_bkg_all[uni+100].SetBinContent(bin_counter,h_temp.GetBinContent(m_bin+1))
        h_smearing_tmp = f_all1.Get('h_smearing['+str(uni)+']')
        h_eff_tmp = f_all1.Get('h_eff_all['+str(uni)+']')
        h_eff_tmp.SetBinContent(5,1)
        h_eff_tmp.SetBinContent(10,1)
        smear = FF.plot2arr(h_smearing_tmp,2)
        eff = FF.plot2arr(h_eff_tmp,1)
        for j in range(43):
            for k in range(43):
                smear[j][k] = smear[j][k]*eff[k]
        h_smearing_all.append(FF.arr2plot(smear))
    
    path_all2 = '/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted_improved/pc46_data/sep24/xsec_genie_all_2_2D_sep24/root_output/'
    f_all2 = ROOT.TFile.Open(path_all2+'All_Genie_2D.root', 'read')
    
    for uni in range(100):
        h_bkg_all.append(h_bkg_all_cv.Clone())
        bin_counter=0
        for t_bin in range(len_theta): #len_theta
            h_temp = f_all1.Get('h_overlay['+str(uni)+']['+str(t_bin)+']')
            for m_bin in range(len_mom[t_bin]):
                bin_counter+=1
                h_bkg_all[uni+200].SetBinContent(bin_counter,h_temp.GetBinContent(m_bin+1))
        h_smearing_tmp = f_all1.Get('h_smearing['+str(uni)+']')
        h_eff_tmp = f_all1.Get('h_eff_all['+str(uni)+']')
        h_eff_tmp.SetBinContent(5,1)
        h_eff_tmp.SetBinContent(10,1)
        smear = FF.plot2arr(h_smearing_tmp,2)
        eff = FF.plot2arr(h_eff_tmp,1)
        for j in range(43):
            for k in range(43):
                smear[j][k] = smear[j][k]*eff[k]
        h_smearing_all.append(FF.arr2plot(smear))
    
    path_all3 = '/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted_improved/pc46_data/sep24/xsec_genie_all_3_2D_sep24/root_output/'
    f_all3 = ROOT.TFile.Open(path_all3+'All_Genie_2D.root', 'read')
    
    for uni in range(100):
        h_bkg_all.append(h_bkg_all_cv.Clone())
        bin_counter=0
        for t_bin in range(len_theta): #len_theta
            h_temp = f_all1.Get('h_overlay['+str(uni)+']['+str(t_bin)+']')
            for m_bin in range(len_mom[t_bin]):
                bin_counter+=1
                h_bkg_all[uni+300].SetBinContent(bin_counter,h_temp.GetBinContent(m_bin+1))
        h_smearing_tmp = f_all1.Get('h_smearing['+str(uni)+']')
        h_eff_tmp = f_all1.Get('h_eff_all['+str(uni)+']')
        h_eff_tmp.SetBinContent(5,1)
        h_eff_tmp.SetBinContent(10,1)
        smear = FF.plot2arr(h_smearing_tmp,2)
        eff = FF.plot2arr(h_eff_tmp,1)
        for j in range(43):
            for k in range(43):
                smear[j][k] = smear[j][k]*eff[k]
        h_smearing_all.append(FF.arr2plot(smear))
    
    path_all4 = '/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted_improved/pc46_data/sep24/xsec_genie_all_4_2D_sep24/root_output/'
    f_all4 = ROOT.TFile.Open(path_all4+'All_Genie_2D.root', 'read')
    
    for uni in range(100):
        h_bkg_all.append(h_bkg_all_cv.Clone())
        bin_counter=0
        for t_bin in range(len_theta): #len_theta
            h_temp = f_all1.Get('h_overlay['+str(uni)+']['+str(t_bin)+']')
            for m_bin in range(len_mom[t_bin]):
                bin_counter+=1
                h_bkg_all[uni+400].SetBinContent(bin_counter,h_temp.GetBinContent(m_bin+1))
        h_smearing_tmp = f_all1.Get('h_smearing['+str(uni)+']')
        h_eff_tmp = f_all1.Get('h_eff_all['+str(uni)+']')
        h_eff_tmp.SetBinContent(5,1)
        h_eff_tmp.SetBinContent(10,1)
        smear = FF.plot2arr(h_smearing_tmp,2)
        eff = FF.plot2arr(h_eff_tmp,1)
        for j in range(43):
            for k in range(43):
                smear[j][k] = smear[j][k]*eff[k]
        h_smearing_all.append(FF.arr2plot(smear))
    
    path_all5 = '/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted_improved/pc46_data/sep24/xsec_genie_all_5_2D_sep24/root_output/'
    f_all5 = ROOT.TFile.Open(path_all5+'All_Genie_2D.root', 'read')
    
    for uni in range(100):
        h_bkg_all.append(h_bkg_all_cv.Clone())
        bin_counter=0
        for t_bin in range(len_theta): #len_theta
            h_temp = f_all1.Get('h_overlay['+str(uni)+']['+str(t_bin)+']')
            for m_bin in range(len_mom[t_bin]):
                bin_counter+=1
                h_bkg_all[uni+500].SetBinContent(bin_counter,h_temp.GetBinContent(m_bin+1))
        h_smearing_tmp = f_all1.Get('h_smearing['+str(uni)+']')
        h_eff_tmp = f_all1.Get('h_eff_all['+str(uni)+']')
        h_eff_tmp.SetBinContent(5,1)
        h_eff_tmp.SetBinContent(10,1)
        smear = FF.plot2arr(h_smearing_tmp,2)
        eff = FF.plot2arr(h_eff_tmp,1)
        for j in range(43):
            for k in range(43):
                smear[j][k] = smear[j][k]*eff[k]
        h_smearing_all.append(FF.arr2plot(smear))
    
    ### flux ############################################
    #path = '/home/tmettler/Desktop/ub_data/mcc9.1/v08_00_00_33/V08_00_00_35/Flux/zarko/'
    #path_flux = '/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted_improved/pc46_data/'

    #weight_list = [ 'expskin_FluxUnisim', 'horncurrent_FluxUnisim' ]
    #weight_list = [ 'expskin_FluxUnisim', 'horncurrent_FluxUnisim', 'kminus_PrimaryHadronNormalization', 'kplus_PrimaryHadronFeynmanScaling',\
    #    'kzero_PrimaryHadronSanfordWang', 'nucleoninexsec_FluxUnisim', 'nucleonqexsec_FluxUnisim', 'nucleontotxsec_FluxUnisim',\
    #    'piminus_PrimaryHadronSWCentralSplineVariation', 'pioninexsec_FluxUnisim', 'pionqexsec_FluxUnisim', 'piontotxsec_FluxUnisim',\
    #    'piplus_PrimaryHadronSWCentralSplineVariation' ]

    #f_flux_int = ROOT.TFile.Open(path+"MCC9_FluxHist_volTPCActive.root", 'read')

    
    
    
    ### save all ########################################
    #save all whats needed
    RootFile = ROOT.TFile('/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted_improved/My_Measurement/'+"FF_detsys.root","RECREATE");
    #blbla.Write('dfdfs')
    h_data.Write('h_data')
    h_ext.Write('h_ext_cv')
    h_dirt.Write('h_dirt_cv')
    h_bkg.Write('h_bkg_cv')

    h_sel.Write('h_sel_cv')
    h_gen.Write('h_gen_cv')
    h_eff.Write('h_eff_cv')
    
    h_smearing.Write('h_smear_cv')
    
    h_smearing_nocrt.Write('h_smear_cv_nocrt')
    h_dirt_nocrt.Write('h_dirt_cv_nocrt')
    h_bkg_nocrt.Write('h_bkg_cv_nocrt')
    
    h_bkg_detcv.Write('h_bkg_detcv')
    h_bkg_dedx.Write('h_bkg_dedx')
    h_bkg_LYatt.Write('h_bkg_LYatt')
    h_bkg_LYdown.Write('h_bkg_LYdown')
    h_bkg_LYray.Write('h_bkg_LYray')
    h_bkg_recomb2.Write('h_bkg_recomb2')
    h_bkg_sce.Write('h_bkg_sce')
    h_bkg_waxz.Write('h_bkg_waxz')
    h_bkg_wayz.Write('h_bkg_wayz')
    h_bkg_wmx.Write('h_bkg_wmx')
    h_bkg_wmyz.Write('h_bkg_wmyz')

    h_smear_detcv.Write('h_smear_detcv')
    h_smear_dedx.Write('h_smear_dedx')
    h_smear_LYatt.Write('h_smear_LYatt')
    h_smear_LYdown.Write('h_smear_LYdown')
    h_smear_LYray.Write('h_smear_LYray')
    h_smear_recomb2.Write('h_smear_recomb2')
    h_smear_sce.Write('h_smear_sce')
    h_smear_waxz.Write('h_smear_waxz')
    h_smear_wayz.Write('h_smear_wayz')
    h_smear_wmx.Write('h_smear_wmx')
    h_smear_wmyz.Write('h_smear_wmyz')

    for w in w_list:
        h_bkg_other[0][w].Write('h_bkg_other['+str(w)+'][0]')
        h_bkg_other[1][w].Write('h_bkg_other['+str(w)+'][1]')
        h_smear_other[0][w].Write('h_smear_other['+str(w)+'][0]')
        h_smear_other[1][w].Write('h_smear_other['+str(w)+'][1]')
    
    h_bkg_all_cv.Write('h_bkg_all_cv')
    h_smearing_all_cv.Write('h_smearing_all_cv')
    for uni in range(600):
        h_smearing_all[uni].Write('h_smearing_all['+str(uni)+']')
        h_bkg_all[uni].Write('h_bkg_all['+str(uni)+']')
    RootFile.Close()

Warning in <TFile::Append>: Replacing existing TH1: h_dirt (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_bkg (Potential memory leak).


In [39]:
print "test"

test


In [ ]:
#store all histograms needed in one file: bkg, smearing, covariance
if(1):
    path_flux = '/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted_improved/Flux/daq2/root_output/'

    #weight_list = [ 'expskin_FluxUnisim', 'horncurrent_FluxUnisim' ]
    weight_list = [ 'expskin_FluxUnisim', 'horncurrent_FluxUnisim', 'kminus_PrimaryHadronNormalization', 'kplus_PrimaryHadronFeynmanScaling',\
        'kzero_PrimaryHadronSanfordWang', 'nucleoninexsec_FluxUnisim', 'nucleonqexsec_FluxUnisim', 'nucleontotxsec_FluxUnisim',\
        'piminus_PrimaryHadronSWCentralSplineVariation', 'pioninexsec_FluxUnisim', 'pionqexsec_FluxUnisim', 'piontotxsec_FluxUnisim',\
        'piplus_PrimaryHadronSWCentralSplineVariation' ]

    f_flux_smear = []
    f_flux_bkg = []
    f_flux = []
    for i,x in enumerate(weight_list):
        f_flux.append(ROOT.TFile.Open(path_flux+x+'_Flux_systematic.root', 'read'))

    h_bkg_cv = f_flux[0].Get('h_bkg_cv')    
    h_smear_cv = f_flux[0].Get('h_smearing_cv')
    
    num_bins = h_smear_cv.GetNbinsX()

    num_universe = 1000
    
    h_bkg_flux = []
    h_smear_flux = []
    for i,x in enumerate(weight_list):
        h_bkg_flux.append([])
        h_smear_flux.append([])
        for uni in range(num_universe):
            #do something
            h_bkg_flux[i].append(f_flux[i].Get('h_bkg_all['+str(uni)+']'))
            h_smear_flux[i].append(f_flux[i].Get('h_smearing['+str(uni)+']'))

    ### save all ########################################
    #save all whats needed
    RootFile = ROOT.TFile('/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted_improved/My_Measurement/'+"FF_flux.root","RECREATE");
    #blbla.Write('dfdfs')
    
    h_bkg_cv.Write('h_bkg_flux_cv')
    h_smear_cv.Write('h_smear_flux_cv')
    
    for i,x in enumerate(weight_list):
        for uni in range(num_universe):
            h_bkg_flux[i][uni].Write('h_bkg_flux['+str(i)+']['+str(uni)+']')
            h_smear_flux[i][uni].Write('h_smear_flux['+str(i)+']['+str(uni)+']')
            
    RootFile.Close()
    


In [9]:
print 'finish'

finish


In [7]:
#store all histograms needed in one file: bkg, smearing, covariance
if(1):
    path = '/home/tmettler/Desktop/ub_data/mcc9.1/v08_00_00_33/V08_00_00_35/Flux/zarko/'

    #weight_list = [ 'expskin_FluxUnisim', 'horncurrent_FluxUnisim' ]
    weight_list = [ 'expskin_FluxUnisim', 'horncurrent_FluxUnisim', 'kminus_PrimaryHadronNormalization', 'kplus_PrimaryHadronFeynmanScaling',\
        'kzero_PrimaryHadronSanfordWang', 'nucleoninexsec_FluxUnisim', 'nucleonqexsec_FluxUnisim', 'nucleontotxsec_FluxUnisim',\
        'piminus_PrimaryHadronSWCentralSplineVariation', 'pioninexsec_FluxUnisim', 'pionqexsec_FluxUnisim', 'piontotxsec_FluxUnisim',\
        'piplus_PrimaryHadronSWCentralSplineVariation' ]
    
    f_flux_int = ROOT.TFile.Open(path+"MCC9_FluxHist_volTPCActive.root", 'read')

    #NumberOfFiles = 5000
    #POT_file = 5e8
    #activeVol_Area = 128.175*2*116.5*2
    
    h_flux_cv = f_flux_int.Get("hEnumu_cv")
    #flux_cv = h_flux_cv.Integral(-1,201)/(NumberOfFiles*POT_file*activeVol_Area)*this_pot

    num_universe = 1000
    
    h_flux_flux = []
    for i,x in enumerate(weight_list):
        h_flux_flux.append([])
        for uni in range(num_universe):
            #do something
            h_flux_flux[i].append(f_flux_int.Get('numu_ms_'+x+'/hEnumu_'+x+'_ms_'+str(uni)))
            
    ### save all ########################################
    #save all whats needed
    RootFile = ROOT.TFile('/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted_improved/My_Measurement/'+"FF_neutrino_flux.root","RECREATE");
    
    h_flux_cv.Write('h_flux_cv')
    for i,x in enumerate(weight_list):
        for uni in range(num_universe):
            h_flux_flux[i][uni].Write('h_flux_flux['+str(i)+']['+str(uni)+']')            
    RootFile.Close()
    


In [8]:
print 'finish'

finish
